In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import cv2
from skimage.feature import local_binary_pattern
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

## Erstelle Augmented Bilder

In [ ]:
# Define the path to the dataset folder
data_folder = "Bilder"

# Define the new subfolder name for augmented images
new_subfolder = "Augmented"

# Define the path to the dataset folder
data_folder = "Bilder"

# Define the new subfolder name for augmented images
new_subfolder = "Augmented"

# Iterate over the folders in the dataset folder
for folder_name in os.listdir(data_folder):
    folder_path = os.path.join(data_folder, folder_name)
    # Check if the item is a folder
    if os.path.isdir(folder_path):
        # Create a new subfolder for augmented images
        new_folder_path = os.path.join(folder_path, new_subfolder)
        os.makedirs(new_folder_path, exist_ok=True)
        # Iterate over the files in the folder
        for file_name in os.listdir(folder_path):
            # Assuming the files are image files, you can adjust the file type according to your dataset
            # Read the image using OpenCV
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(folder_path, file_name)
                image = cv2.imread(image_path)
                # Check if the image is loaded successfully
                if image is not None:
                    # Perform image augmentation
                    # Resize the image to a desired size
                    resized_image = cv2.resize(image, (300, 300))
                    datagen = ImageDataGenerator(
                        rotation_range=40,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip=True,
                        fill_mode='nearest'
                    )
                    # Reshape the image to add batch dimension
                    reshaped_image = resized_image.reshape((1,) + resized_image.shape)
                    # Generate and save augmented images
                    i = 0
                    for batch in datagen.flow(reshaped_image, batch_size=1, save_to_dir=new_folder_path, save_prefix='aug', save_format='png'):
                        i += 1
                        if i >= 10:
                            break  # Break the loop after generating 100 augmented images
                else:
                    print(f"Failed to load image: {image_path}")
            else:
                print(f"Skipping non-image file: {file_name}")
           

            

## Vektorize Bilder und erstelle Labels dazu

In [ ]:
# Initialize lists to store images and labels
images = []
labels = []

# Iterate over the folders in the dataset folder
for folder_name in os.listdir(data_folder):
    folder_path = os.path.join(data_folder, folder_name)
    # Check if the item is a folder
    if os.path.isdir(folder_path):
        # Get the label from the folder name
        label = folder_name
        # Get the path to the augmented folder
        augmented_folder_path = os.path.join(folder_path, new_subfolder)
        # Check if the augmented folder exists
        if os.path.exists(augmented_folder_path):
            # Iterate over the files in the augmented folder
            for file_name in os.listdir(augmented_folder_path):
                # Read the image using OpenCV
                image_path = os.path.join(augmented_folder_path, file_name)
                image = cv2.imread(image_path)
                # Check if the image is loaded successfully
                if image is not None:
                    # Add the image to the list of images
                    images.append(image)
                    # Add the label to the list of labels
                    labels.append(label)
                else:
                    print(f"Failed to load image: {image_path}")

print(f"Total images: {len(images)}")
print(f"Total labels: {len(labels)}")



## Hog Feature Extraction

In [ ]:
from skimage.feature import hog
import cv2

# Initialize empty lists to store HOG features and labels
hog_features = []
hog_labels = []

# Iterate over the folders in the dataset folder
for folder_name in os.listdir(data_folder):
    folder_path = os.path.join(data_folder, folder_name)
    # Check if the item is a folder
    if os.path.isdir(folder_path):
        # Get the path to the augmented folder
        augmented_folder_path = os.path.join(folder_path, new_subfolder)
        # Check if the augmented folder exists
        if os.path.exists(augmented_folder_path):
            # Iterate over the files in the augmented folder
            for file_name in os.listdir(augmented_folder_path):
                # Read the image using OpenCV
                image_path = os.path.join(augmented_folder_path, file_name)
                image = cv2.imread(image_path)
                # Check if the image is loaded successfully
                if image is not None:
                    # Convert the image to grayscale
                    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    # Perform HOG feature extraction
                    hog_feature = hog(gray_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)
                    # Add the HOG feature to the list of features
                    hog_features.append(hog_feature)
                    # Add the label to the list of labels
                    hog_labels.append(folder_name)
                    # Print image filename, HOG features, and label
                    print(f"Image: {file_name}, HOG features: {hog_feature}, Label: {folder_name}")
                else:
                    print(f"Failed to load image: {image_path}")

print(f"Total HOG features: {len(hog_features)}")
print(f"Total labels: {len(labels)}")


## Feature Extraction functions

In [ ]:
def extract_lbp_features(image, num_points, radius):
    lbp_features = []
    for channel in range(image.shape[2]):  # Iteriere über die Farbkanäle (RGB)
        lbp = local_binary_pattern(image[:, :, channel], num_points, radius, method='uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, num_points + 3), range=(0, num_points + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-7)
        lbp_features.extend(hist)
    return lbp_features

def predict_image_LBP(image_path, classifier, num_points, radius):
    # Lade das Bild
    image = cv2.imread(image_path)
    if image is not None:
        # Verkleinere das Bild
        resized_image = cv2.resize(image, (300, 300))
        # Extrahiere die LBP-Merkmale
        lbp_features = extract_lbp_features(resized_image, num_points, radius)
        # Führe die Vorhersage mit dem Klassifikator durch
        prediction = classifier.predict([lbp_features])[0]
        # Bestimme die Vertrauenswahrscheinlichkeit für die Vorhersage
        confidence = max(classifier.predict_proba([lbp_features])[0])
        return prediction, confidence
    else:
        print(f"Failed to load image: {image_path}")
        return None, None

## LBP Feature Extraction

In [ ]:


# Initialize empty list to store LBP features
lbp_features = []
lbp_labels = []
 
# Parameter für LBP festlegen
num_points = 24
radius = 2

for folder_name in os.listdir(data_folder):
    folder_path = os.path.join(data_folder, folder_name)
    # Check if the item is a folder
    if os.path.isdir(folder_path):
        # Get the path to the augmented folder
        augmented_folder_path = os.path.join(folder_path, new_subfolder)
        # Check if the augmented folder exists
        if os.path.exists(augmented_folder_path):
            # Iterate over the files in the augmented folder
            for file_name in os.listdir(augmented_folder_path):
                # Read the image using OpenCV
                image_path = os.path.join(augmented_folder_path, file_name)
                image = cv2.imread(image_path)
                # Check if the image is loaded successfully
                if image is not None:
                    lbp_feature = extract_lbp_features(image, num_points, radius)
                    lbp_features.append(lbp_feature)
                    # Add the label to the list of labels
                    lbp_labels.append(folder_name)
                    print(f"Image: {file_name}, LBP features: {lbp_feature}, Label: {folder_name}")
                else:
                    print(f"Failed to load image: {image_path}")


## Random Forest

In [ ]:
from sklearn.model_selection import cross_val_score

# Aufteilung der Daten in Trainings- und Testsets
X_train, X_test, y_train, y_test = train_test_split(lbp_features, labels, test_size=0.2, random_state=42)
 
# Initialisierung des Random-Forest-Klassifikators
rf_classifier = RandomForestClassifier(n_estimators=10, random_state=42, n_jobs=-1, verbose=1 , max_depth=10)
 
# Durchführung der Kreuzvalidierung mit 5 Faltungen
cv_scores = cross_val_score(rf_classifier, X_train, y_train, cv=5)

# Trainieren des Random-Forest-Modells auf dem gesamten Trainingsdatensatz
rf_classifier.fit(X_train, y_train)
 
# Evaluierung des Modells auf dem Testdatensatz (optional)
accuracy = rf_classifier.score(X_test, y_test)
predicted = rf_classifier.predict(X_test)

# Ausgabe der Ergebnisse
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy (Cross-Validation):", cv_scores.mean())
print("Accuracy (Test Set):", accuracy)
print("Predicted (Test Set):", predicted)
 

## Bilder predicten

In [ ]:
# Ordnerpfad mit den zu klassifizierenden Bildern
predict_folder = "predict"

# Iteriere über die Bilder im Ordner und führe die Vorhersage durch
for filename in os.listdir(predict_folder):
    image_path = os.path.join(predict_folder, filename)
    prediction, confidence = predict_image_LBP(image_path, rf_classifier, num_points, radius)
    if prediction is not None:
        print(f"Prediction for {filename}: {prediction}, Confidence: {confidence}")